In [99]:
import atoti as tt

In [100]:
session = tt.Session(
    user_content_storage=".content",
    port=9095,
    java_options=["-Xms1G", "-Xmx10G"]
)

Closing existing "Unnamed" session to create the new one.


In [101]:
db_name = "Olympic"
db_user = "postgres"
db_password = "postgres"  
db_host = "localhost"  
db_port = "5432"

In [102]:
jdbc_url = f"jdbc:postgresql://{db_host}:{db_port}/{db_name}?user={db_user}&password={db_password}"

In [103]:
jdbc_url

'jdbc:postgresql://localhost:5432/Olympic?user=postgres&password=postgres'

In [104]:
facttest = session.read_sql(
    "SELECT * FROM factolympic",
    keys=["year","country"],
    table_name="factTable",
    url=jdbc_url,
)

In [160]:
time.head()

,decade,season
year,,
2020,2020 - 2029,Summer
2004,2000 - 2009,Summer
1984,1980 - 1989,Winter
1952,1950 - 1959,Winter
1908,1900 - 1909,Summer


In [106]:
# medals = session.read_sql(
#     "SELECT * FROM medalsnew",
#     keys=["id"],
#     table_name="Medals",
#     url=jdbc_url)

In [107]:
countries = session.read_sql(
    "SELECT * FROM countries",
    keys=["country"],
    table_name="Countries",
    url=jdbc_url)

In [108]:
economic = session.read_sql(
    "SELECT * FROM economic",
    keys=["country"],
    table_name="Economic",
    url=jdbc_url)

In [109]:
time = session.read_sql(
    "SELECT * FROM timenew",
    keys=["year"],
    table_name="Time",
    url=jdbc_url)

In [110]:
lifeexp = session.read_sql(
    "SELECT * FROM lifeexp",
    keys=["year","country"],
    table_name="LifeExp",
    url=jdbc_url)

In [111]:
facttest.join(countries, facttest["country"] == countries["country"])

In [112]:
facttest.join(economic, facttest["year"] == economic["year"])

In [113]:
facttest.join(time, facttest["year"] == time["year"])

In [114]:
facttest.join(lifeexp, (facttest["year"] == lifeexp["year"] )& (facttest["country"] == lifeexp["country"]))

In [115]:
session.tables.schema

```mermaid
erDiagram
  "Economic" {
    nullable int "year"
    _ String "yearcode"
    _ String PK "country"
    _ String "code"
    nullable double "poverty"
    nullable double "gdpcap"
    nullable double "gdpgrowth"
    nullable double "intsrv"
    nullable double "mort"
    nullable double "hlthexp"
    nullable double "govhlthcap"
    nullable double "prvhlthcap"
    nullable double "exthlthcap"
  }
  "LifeExp" {
    _ String PK "country"
    _ String "code"
    _ int PK "year"
    nullable double "lifeexpectancy"
  }
  "Countries" {
    _ String PK "country"
    _ String "continent"
  }
  "Time" {
    _ int PK "year"
    _ String "decade"
    _ String "season"
  }
  "factTable" {
    _ int PK "year"
    _ String PK "country"
    nullable int "bronze"
    nullable int "silver"
    nullable int "gold"
    nullable int "total"
    nullable double "depression"
    nullable double "schizophrenia"
    nullable double "bipolar"
    nullable double "eatingdisorder"
    nullable double "anxiety"
  }
  "factTable" }o--o| "Countries" : "`country` == `country`"
  "factTable" }o--o| "LifeExp" : "(`country` == `country`) & (`year` == `year`)"
  "factTable" }o--o| "Time" : "`year` == `year`"
  "factTable" }o..o{ "Economic" : "`year` == `year`"
```


In [116]:
testcube = session.create_cube(facttest)

In [117]:
testcube

In [118]:
hierarchies, levels, measures = testcube.hierarchies, testcube.levels, testcube.measures

In [127]:
hierarchies

{('Time', 'year'): <atoti.hierarchy.Hierarchy object at 0x72b62c205cf0>, ('Countries', 'continent'): <atoti.hierarchy.Hierarchy object at 0x72b62c2040a0>, ('Time', 'decade'): <atoti.hierarchy.Hierarchy object at 0x72b62c204310>, ('LifeExp', 'code'): <atoti.hierarchy.Hierarchy object at 0x72b62c2044c0>, ('factTable', 'year'): <atoti.hierarchy.Hierarchy object at 0x72b62c2045b0>, ('factTable', 'country'): <atoti.hierarchy.Hierarchy object at 0x72b62c2045e0>, ('Time', 'season'): <atoti.hierarchy.Hierarchy object at 0x72b62c2046d0>, ('Economic', 'code'): <atoti.hierarchy.Hierarchy object at 0x72b62c204880>, ('Economic', 'country'): <atoti.hierarchy.Hierarchy object at 0x72b62c3d7790>, ('Countries', 'country'): <atoti.hierarchy.Hierarchy object at 0x72b62c3d6140>}

In [140]:
list(levels)

[('Time', 'year', 'year'),
 ('Time', 'year', 'decade'),
 ('Time', 'year', 'season'),
 ('Countries', 'country', 'country'),
 ('Countries', 'country', 'continent')]

In [180]:
hierarchies["Time","year"] = [time["decade"],time["season"],time["year"]]

In [139]:
del hierarchies["Economic","country"]

In [179]:
list(testcube.levels)

[('Time', 'year', 'decade'),
 ('Time', 'year', 'year'),
 ('Time', 'year', 'season'),
 ('Countries', 'country', 'continent'),
 ('Countries', 'country', 'country')]

In [181]:
testcube.query(measures["bronze.SUM"],measures["silver.SUM"],measures["gold.SUM"],measures["depression.SUM"], levels=[levels["Time","year","year"],levels['Countries', 'country', 'country']])

bronze.SUM silver.SUM  \
decade      season year continent     country                              
1990 - 1999 Summer 1996 Africa        Algeria               1          0   
                                      Angola                0          0   
                                      Benin                 0          0   
                                      Botswana              0          0   
                                      Burkina Faso          0          0   
...                                                       ...        ...   
2010 - 2019 Winter 2018 South America Paraguay              0          0   
                                      Peru                  0          0   
                                      Suriname              0          0   
                                      Uruguay               0          0   
                                      Venezuela             0          0   

                                                   gold.SUM depression.SUM  
decade      season year continent     country                               
1990 - 1999 Summer 1996 Africa        Algeria             2         749.54  
                                      Angola              0       1,087.57  
                                      Benin               0         750.43  
                                      Botswana            0         725.62  
                                      Burkina Faso        0         725.99  
...                                                     ...            ...  
2010 - 2019 Winter 2018 South America Paraguay            0         666.39  
                                      Peru                0         353.82  
                                      Suriname            0         876.10  
                                      Uruguay             0         511.06  
                                      Venezuela           0         567.80  

[2772 rows x 4 columns]

In [124]:
testcube.query(measures["total.SUM"],measures["depression.MEAN"], levels = [levellevels["Countries",'country','country']])

,total.SUM,depression.MEAN
country,,
Afghanistan,2,891.18
Albania,0,381.48
Algeria,15,736.32
American Samoa,0,391.00
Andorra,0,621.74
...,...,...
Venezuela,7,566.37
Vietnam,4,372.64
Yemen,0,892.93
